# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
exported_data = "../starter_code/City_Data.csv"
weather_df = pd.read_csv(exported_data)
weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,São João da Barra,-21.64,-41.05,296.77,89,71,6.83
1,1,Ushuaia,-54.80,-68.30,281.15,39,40,3.60
2,2,Avarua,-21.21,-159.78,296.15,78,100,3.10
3,3,Lavrentiya,65.58,-171.00,272.34,91,100,4.41
4,4,Lebu,-37.62,-73.65,283.55,82,98,5.50
...,...,...,...,...,...,...,...,...
548,548,Bonnyville,54.27,-110.73,280.15,65,75,7.20
549,549,Black Diamond,47.31,-122.00,287.04,69,81,0.64
550,550,Noyabrsk,63.20,75.45,257.56,93,20,6.68
551,551,Chimbote,-9.09,-78.58,290.16,88,90,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Latitude", "Longitude"]]
humid = weather_df["Humidity"]
h_map = gmaps.figure(center=(50, -10), zoom_level=2)
max_intensity = np.max(humid)
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False, max_intensity=100, point_radius=3)
h_map.add_layer(heat_layer)
h_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
fit_city_df = weather_df.loc[(weather_df["Temperature"] >= 280) & (weather_df["Temperature"] <= 300) & (weather_df["Cloudiness"] ==0) & (weather_df["Wind Speed"] <=1)].dropna()
fit_city_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
49,49,Souillac,-20.52,57.52,294.26,80,0,0.89
79,79,Rāwatbhāta,24.93,75.58,292.58,26,0,0.53
112,112,Keïta,14.76,5.77,298.15,29,0,1.00
120,120,Asău,46.43,26.40,283.15,98,0,0.77
231,231,Cap Malheureux,-19.98,57.61,294.26,80,0,0.89
256,256,Diego de Almagro,-26.37,-70.05,285.42,58,0,0.68
283,283,Kota,25.18,75.83,293.31,25,0,0.96
330,330,Tabas,33.60,56.92,288.15,26,0,1.00
387,387,San Vicente,-26.62,-54.13,288.76,72,0,0.32
396,396,Valls,41.29,1.25,287.04,71,0,1.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
hotel_df = fit_city_df.loc[:,["City", "Latitude", "Longitude"]]
hotel_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Hotel Name
49,Souillac,-20.52,57.52,
79,Rāwatbhāta,24.93,75.58,
112,Keïta,14.76,5.77,
120,Asău,46.43,26.40,
231,Cap Malheureux,-19.98,57.61,
256,Diego de Almagro,-26.37,-70.05,
283,Kota,25.18,75.83,
330,Tabas,33.60,56.92,
387,San Vicente,-26.62,-54.13,
396,Valls,41.29,1.25,


In [77]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
hotel_df

,City,Latitude,Longitude,Hotel Name
49,Souillac,-20.52,57.52,SkylineInn Hotel
79,Rāwatbhāta,24.93,75.58,Bhainsrorgarh Fort Hotel
112,Keïta,14.76,5.77,
120,Asău,46.43,26.40,Pensiunea Sophia
231,Cap Malheureux,-19.98,57.61,Paradise Cove Boutique Hotel
256,Diego de Almagro,-26.37,-70.05,hotel luna del desierto
283,Kota,25.18,75.83,WelcomHeritage Umed Bhawan Palace
330,Tabas,33.60,56.92,Amir hotel
387,San Vicente,-26.62,-54.13,Hotel Y Restaurante Los Lapachos
396,Valls,41.29,1.25,FELIX HOTEL


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [79]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
h_map.add_layer(markers)

# Display figure


NameError: name 'hotel_info' is not defined